# Producer Development


### Notebook for testing out the interface to Twitter and IEX

In [138]:
#Import
import numpy as np
import pandas as pd

#Twitter requirements
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

#IEX requirements
import urllib.request


import json

### Create Company List

In [139]:
companies = {
    "AAPL":"Apple",
    "FB":"Facebook",
    "GOOG":"Google"
    }

#company_df = pd.DataFrame(companies.items(), columns=['Company Name','Ticker'])
company_df = pd.DataFrame.from_dict(companies, orient='index')
company_df.index.name = 'Ticker'
company_df.columns=['Company']
company_df.head()

,Company
Ticker,
AAPL,Apple
FB,Facebook
GOOG,Google


In [91]:
#Write to CSV
company_df.to_csv('companies.csv')

### Twitter Ingestion

In [140]:
#Created a twitter tokens file. That has the tokens. This will not be in GIT
from twitter_tokens import *

In [141]:
#Add Code to read CSV file. In order to account for changes in the list, have to restart
companies = pd.read_csv('companies.csv')
companies.set_index('Ticker', inplace=True)

#Add code to add ticker symbol
companies['tweet_ticker']=companies.index.map(lambda x: '$'+x)
companies

,Company,tweet_ticker
Ticker,,
AAPL,Apple,$AAPL
FB,Facebook,$FB
GOOG,Google,$GOOG


In [119]:
#tickers = companies['tweet_ticker'].tolist()
tickers = companies['tweet_ticker'].tolist()
#Tweets with the ticker in front come in very slowly. May take a while to build up. But these are official tweets

In [142]:
#Create Filter function to filter attributes and Add Company name to the dictionary.

#Want to create a simple list, but how do I handle the users sub?
attributes = ['created_at',
             'id_str',
             'text',
              'quote_count',
              'reply_count',
              'retweet_count',
              'favorite_count',
              'retweeted',
              ['user','name'],
              ['user','followers_count'],
              ['user','statuses_count'],
              ['user','screen_name'],
              ['entities','hashtags'],
              ['entities','symbols']
               ]
def filter_attr(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        if isinstance(element, str):
            output[element]=data[element]
        else:
            string = str(element[0])+'_'+str(element[1])
            output[string]=data[element[0]][element[1]]
  
    #Need to also add the company name to output dictionary.
    #Add all companies tweet applies to in list
    attached_company = []
    
    for company in tickers:
        if data['text'].find(company) > -1:
            attached_company.append(company[1:])
            
    
    output['Company']=attached_company             
    
    return output

#This is a basic listener that just prints received tweets to stdout.
class TweetListener(StreamListener):
    global count
    count = 0
    def on_data(self, data):
        try:
            datajson = json.loads(data)
            filtered = filter_attr(datajson)
            #Check to see if a valid tweet
            if filtered['Company']:
                global count
                count += 1
                print(count)
                print(filtered)
            #print(datajson['text'])
            #coll_reference.insert_one(datajson)
            return True
        except Exception as e:
            print(e)

    def on_error(self, status):
        print('The error code is: ' + repr(status))
        #Continue even if there is an error
        return True

#This handles Twitter authetication and the connection to Twitter Streaming API
tweetlist = TweetListener(api=API(wait_on_rate_limit=True,wait_on_rate_limit_notify=True))
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
stream = Stream(auth, tweetlist)

#Filters by the ticker names
print('Filtering: ' + str(tickers))
stream.filter(track=tickers)

Filtering: ['$AAPL', '$FB', '$GOOG']
1
{'created_at': 'Sun Jan 28 03:08:48 +0000 2018', 'id_str': '957450356350439424', 'text': 'RT @OMillionaires: Some implied moves for next week earnings, it is a big one:\n\n$AAPL 4.7%\n$BABA 6.9%\n$AMZN 6.8%\n$FB 5.8%\n$GOOGL 5%\n$MSFT 5…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'user_name': 'Nosson Herzka', 'user_followers_count': 1570, 'user_statuses_count': 24813, 'user_screen_name': 'nosmh', 'entities_hashtags': [], 'entities_symbols': [{'text': 'AAPL', 'indices': [80, 85]}, {'text': 'BABA', 'indices': [91, 96]}, {'text': 'AMZN', 'indices': [102, 107]}, {'text': 'FB', 'indices': [113, 116]}, {'text': 'GOOGL', 'indices': [122, 128]}, {'text': 'MSFT', 'indices': [132, 137]}], 'Company': ['AAPL', 'FB', 'GOOG']}


KeyboardInterrupt: 